In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import random

weather_csv = gpd.read_file("./기상청_종관기상관측_ASOS.csv", encoding='cp949')
accident_csv = gpd.read_file("제주특별자치도_수난사고 구조출동 현황_2011_2020.csv")
length = len(accident_csv)
ran = int(random.random() * length)
print("coordinate : " + str(accident_csv['loc_info_x'][ran]) + ", " + str(accident_csv['loc_info_y'][ran]))

coordinate : 159481.78, 53738.59


In [2]:
'''
url = 'https://kauth.kakao.com/oauth/token'
rest_api_key = '359e276820f89feab230a8eb1eeadedd'
redirect_uri = 'http://localhost:8888/' # APP에서 등록한 redirect_url
authorize_code = '-Ti1X-TEIwWEg8lHx3xTPOKZk5TdUQMW4M_joY99Vw_GMUHg78lt7g-McQx5fhPTvvUt0worDR4AAAGCFOaPpA'
 
data = {
        'grant_type':'authorization_code',
        'client_id':rest_api_key,
        'redirect_uri':redirect_uri,
        'code': authorize_code,
    }

response = requests.post(url, data=data)
tokens = response.json()
print(tokens["access_token"])
'''

NameError: name 'requests' is not defined

In [2]:
import requests

new_idx = []
new_x = []
new_y = []

for idx in range(0, length):  
    if(str(accident_csv['loc_info_y'][idx]) != ''):
        method = "GET"
        url = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
        params = {'x' : str(accident_csv['loc_info_x'][idx]), 
                  'y' : str(accident_csv['loc_info_y'][idx]),
                  'input_coord' : 'TM',
                  'output_coord' : 'WGS84'}
        header = {'authorization': 'KakaoAK 359e276820f89feab230a8eb1eeadedd'}

        response = requests.request(method=method, url=url, headers=header, params=params )
        tokens = response.json()

        x = tokens['documents'][0]['x']
        y = tokens['documents'][0]['y'] - 0.453

        new_idx.append(accident_csv['resc_reprt_no'])
        new_x.append(x)
        new_y.append(y)
    else: #has no coordinate
        print(idx)

88


In [3]:
#only one

import requests

method = "GET"
url = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
params = {'x' : 139516.8018, 
          'y' : 21483.4872,
          'input_coord' : 'WTM',
          'output_coord' : 'WGS84'}
header = {'authorization': 'KakaoAK 359e276820f89feab230a8eb1eeadedd'}

response = requests.request(method=method, url=url, headers=header, params=params )
tokens = response.json()

x = tokens['documents'][0]['x']
y = tokens['documents'][0]['y'] - 0.5

x,y
#input_coord 해본것들 WKTM, WTM(Y좌표만 어떻게하면?), TM (이것도 Y만), 

#126.3477 x좌표는 이건맞는듯
#WTM->WGS84 : (126.34771077228292, 33.685619937849175)
#CONGNAMUL->WGS84 : (125.44799755619425, 33.56415669858657)
#WCONGNAMUL->WGS84 : (125.44721978862324, 33.561386137382634)
#KTM->WGS84 : (125.21831790311661, 32.757107783355934)
#KTM->BESSEL : (125.22023064710514, 32.753776835065956)


(126.34771077228292, 33.185619937849175)

In [68]:
!pip install folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 3.0 MB/s eta 0:00:00


In [12]:
import folium
m = folium.Map(location = [33.3734235,126.5512051,], zoom_start=10)
m.save('map.html')

In [9]:
import geopandas as gpd
import pandas as pd
import matplotlib as plt

si = gpd.read_file('./CTPRVN_20220324/ctp_rvn.shp', encoding = 'euc-kr')
si = si.to_crs(epsg=4326)
dong = gpd.read_file("./EMD_20220324/emd.shp", encoding = 'euc-kr')
dong = dong.to_crs(epsg=4326)
si = si[si['CTP_ENG_NM'] == 'Jeju-do']

dropList = []
for i in range(0, len(dong)):
    if(si['geometry'].intersects(dong['geometry'][i]).bool() == False):
        dropList.append(i)

dong = dong.drop(dropList)

east_list = ["구좌읍", "우도면", "표선면", "성산읍"]
west_list = ["한림읍", "한경면", "대정읍"]
north_list = ["애월읍", "조천읍", "해안동", "봉개동", "회천동", "월평동", "영평동", "연동", "노형동", "용강동", "오라이동", "오등동", "아라일동", "아라이동", "화북일동", "화북이동", "도련일동", "도련이동", "삼양일동", "삼양이동", "삼양삼동", "일도이동", "이도이동", "오라일동", "오라이동", "오라삼동", "건입동", "도남동", "용담일동", "용담이동", "용담삼동", "도두일동", "도두이동", "삼도일동", "삼도이동", "일도일동", "이도일동", "이도이동", "이호일동", "이호이동", "외도일동", "외도이동", "내도동", "도평동", "추자면"]
south_list = ["안덕면", "남원읍", "상예동", "하예동", "색달동", "중문동", "대포동", "회수동", "하원동", "도순동", "영남동", "강정동", "법환동", "서호동", "호근동", "서홍동", "동홍동", "서귀동", "토평동", "상효동", "보목동", "하효동", "신효동"]

dong['LOCATION'] = ""
for index, r in dong.iterrows():
    if(r['EMD_KOR_NM'] in east_list):
        dong.loc[index, 'LOCATION'] = "EAST"
    elif(r['EMD_KOR_NM'] in west_list):
        dong.loc[index, 'LOCATION'] = "WEST"
    elif(r['EMD_KOR_NM'] in north_list):
        dong.loc[index, 'LOCATION'] = "NORTH"
    elif(r['EMD_KOR_NM'] in south_list):
        dong.loc[index, 'LOCATION'] = "SOUTH"

continents = dong.dissolve(by='LOCATION', aggfunc='sum')
jeju_east = continents['geometry']['EAST']
jeju_west = continents['geometry']['WEST']
jeju_north = continents['geometry']['NORTH']
jeju_south = continents['geometry']['SOUTH']

print(type(jeju_east))


<class 'shapely.geometry.multipolygon.MultiPolygon'>


In [13]:
for _, r in dong.iterrows():

    
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()

    if(r['LOCATION'] == "EAST"):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'green'})
    elif(r['LOCATION'] == "WEST"):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'blue'})
    elif(r['LOCATION'] == "NORTH"):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'red'})
    elif(r['LOCATION'] == "SOUTH"):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'orange'})
    else:
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'black'})

    lat = r['geometry'].centroid.y
    lon = r['geometry'].centroid.x
    #folium.Marker(location=[lat, lon],popup='{}'.format(r['EMD_KOR_NM'])).add_to(m)


    geo_j.add_to(m)
m

In [56]:
east_count = 0
west_count = 0
north_count = 0
south_count = 0
no_where = 0
for i in range(0, len(new_x)):
    point = Point(new_x[i], new_y[i])
    
    if(jeju_east.intersects(point) == True):
        east_count+=1
    elif(jeju_west.intersects(point) == True):
        west_count+=1
    elif(jeju_north.intersects(point) == True):
        north_count+=1
    elif(jeju_south.intersects(point) == True):
        south_count+=1
    else:
        no_where += 1


print("Total Data : " + str(len(new_x)))
print("EAST Accident Count : " + str(east_count))
print("WEST Accident Count : " + str(west_count))
print("SOUTH Accident Count : " + str(south_count))
print("NORTH Accident Count : " + str(north_count))
print("don't know where in Accident is Count : " + str(no_where))
    

Total Data : 1546
EAST Accident Count : 123
WEST Accident Count : 54
SOUTH Accident Count : 258
NORTH Accident Count : 286
don't know where in Accident is Count : 825
